In [1]:
import numpy as np
import pandas as pd

# Vectorizing the revies of users

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorize reviews
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
review_vectors = tfidf.fit_transform(df_matrix['text'])

In [56]:
review_vectors.shape[0]

600000

## Averaging reviews

In [43]:
# Map user_id to numeric index
user_id_map = {uid: idx for idx, uid in enumerate(df_matrix['user_id'].unique())}
df_matrix['user_idx'] = df_matrix['user_id'].map(user_id_map)




In [59]:
user_aggregation=[]
for i in range(review_vectors.shape[0]):
    user_aggregation.append(review_vectors[i].mean())

In [58]:
review_vectors[2].mean()

np.float64(0.0010833878206556902)

In [62]:
user_review_avg=pd.DataFrame(user_aggregation,columns=['user_review_avg'])

In [65]:
user_vendor_series=pd.concat([df_matrix,user_review_avg],axis=1)

In [66]:
user_vendor_series.to_csv("../r2Data/user_vendor_series_data.csv")

In [13]:
user_vendor_data.dropna(inplace=True)

# Convert vendor index to codes

In [17]:
# Map user_id to numeric index
user_id_map = {uid: idx for idx, uid in enumerate(user_vendor_data['business_id'].unique())}
user_vendor_data['vendor_idx'] = user_vendor_data['business_id'].map(user_id_map)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix

# Vectorize user reviews
t1 = TfidfVectorizer(max_features=5000, stop_words='english')
review_vectors = t1.fit_transform(user_vendor_data['text'])




In [18]:
user_vendor_data['user_idx'].nunique()

347940

In [19]:
t2 = TfidfVectorizer(max_features=5000, stop_words='english')
vendor_profile = t2.fit_transform(user_vendor_data['categories'])

In [20]:
from sklearn.preprocessing import normalize

# Normalize user profiles and vendor profiles
user_profiles_normalized = normalize(review_vectors)
vendor_profiles_normalized = normalize(vendor_profile)


In [21]:
review_vectors.shape

(551582, 5000)

In [ ]:
num_users = user_vendor_data['user_idx'].nunique()
user_profiles = csr_matrix((num_users, review_vectors.shape[1]), dtype=np.float64)

for user_idx in range(num_users):
    user_reviews = review_vectors[np.array(user_vendor_data['user_idx'] == user_idx)]
    if user_reviews.shape[0] > 0:
        user_profiles[user_idx] = user_reviews.mean(axis=0)

print("User profiles shape:", user_profiles.shape)

C:\Users\ujwal\wedora_env\wedoraenv\Lib\site-packages\scipy\sparse\_index.py:197: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_arrayXarray(i, j, x)
